In [ ]:
%load_ext lab_black
import json
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import ticker as tck
import seaborn as sns
from wikipedia_cleanup.schema import InfoboxRevision
from typing import Iterable, Tuple, Any, Optional
from ipywidgets import (
    interact,
    IntSlider,
    IntRangeSlider,
    FloatRangeSlider,
    Checkbox,
    ToggleButtons,
)

In [ ]:
sns.set_theme(style="ticks")
cm = 1 / 2.54
a4 = 29.7, 42

In [ ]:
def get_data(n_files: Optional[int] = None) -> Iterable[Tuple[Any, ...]]:
    files = [
        x
        for x in sorted(Path("../matched-infoboxes-extracted/").rglob("*.output.json"))
        if x.is_file()
    ]
    for file in files[slice(n_files)]:
        with open(file) as f:
            for line in f:
                revision = InfoboxRevision(**json.loads(line))
                for change in revision.changes:
                    yield (
                        revision.key,
                        revision.revisionId,
                        revision.pageID,
                        revision.pageTitle,
                        change.property.name,
                        change.previousValue,
                        change.currentValue,
                        revision.validFrom,
                    )

In [ ]:
df = pd.DataFrame(
    get_data(n_files=5),
    columns=[
        "key",
        "revisionId",
        "pageID",
        "pageTitle",
        "property.name",
        "previousValue",
        "currentValue",
        "validFrom",
    ],
)
df["key"] = df["key"].astype(str)
df["revisionId"] = df["revisionId"].astype(int)
df["pageID"] = df["pageID"].astype(int)
df["pageTitle"] = df["pageTitle"].astype(str)
df["property.name"] = df["property.name"].astype(str)
df = df.set_index(["revisionId", "key", "property.name"]).sort_index()
df

In [ ]:
@interact(
    bins=IntSlider(50, 1, 100, 1, description="Bins", continuous_update=False),
    kde=Checkbox(True, description="KDE"),
)
def plot_hist1(bins, kde) -> None:
    data = df.reset_index().groupby("pageID")[["property.name"]].count()
    fig, ax = plt.subplots(figsize=(a4[0] * cm, (a4[0] / 2) * cm), dpi=300)
    sns.histplot(
        data=data,
        stat="percent",
        bins=bins,
        kde=kde,
        log_scale=True,
        legend=False,
        ax=ax,
    )
    ax.set_xlim(left=1)
    ax.yaxis.set_major_formatter(tck.PercentFormatter())
    plt.title("Number of Properties per Page")
    plt.xlabel("Number of Properties")
    plt.ylabel("Percentage of Pages")
    sns.despine(ax=ax)
    plt.show()

In [ ]:
@interact(
    bins=IntSlider(50, 1, 100, 1, description="Bins", continuous_update=False),
    kde=Checkbox(True, description="KDE"),
)
def plot_hist2(bins, kde) -> None:
    data = df.reset_index().groupby("property.name")[["pageID"]].count()
    data["pageID"] = 100 * data["pageID"] / data["pageID"].sum()
    fig, ax = plt.subplots(figsize=(a4[0] * cm, (a4[0] / 2) * cm), dpi=300)
    sns.histplot(
        data=data,
        stat="percent",
        bins=bins,
        log_scale=True,
        kde=kde,
        legend=False,
        ax=ax,
    )
    ax.xaxis.set_major_formatter(tck.PercentFormatter(decimals=4))
    ax.yaxis.set_major_formatter(tck.PercentFormatter())
    plt.title("Prevalence of Properties")
    plt.xlabel("Percentage of Pages")
    plt.ylabel("Percentage of Properties")
    sns.despine(ax=ax)
    plt.show()

In [ ]:
@interact(kind=ToggleButtons(options=["Pie", "Bar"], description="Kind"))
def plot_hist3(kind) -> None:
    k = kind.lower()
    fig, ax = plt.subplots(figsize=(a4[0] * cm, (a4[0] / 2) * cm), dpi=100)
    pd.DataFrame(
        [
            (df["previousValue"].isna()).sum() / len(df["previousValue"]),
            (df["currentValue"].isna()).sum() / len(df["currentValue"]),
            (df["previousValue"].notna() & df["currentValue"].notna()).sum()
            / len(df["currentValue"]),
        ],
        index=["Creation", "Deletion", "Modification"],
        columns=["Percentage"],
    ).sort_values("Percentage", ascending=False).plot(
        y="Percentage", kind=k, ax=ax, legend=False, rot=0
    )
    ax.yaxis.set_major_formatter(tck.PercentFormatter(xmax=1))
    sns.despine(ax=ax)
    plt.title("Changes by Type")
    plt.xlabel("Type of Change")
    plt.ylabel("Percentage of Changes")
    plt.show()

# MVP Model

Predicts the mean days per change

In [ ]:
df2 = (
    df.reset_index()
    .set_index(["pageID", "property.name", "validFrom"])
    .sort_values(["revisionId", "key", "property.name"])
    .groupby(["pageID", "property.name", pd.Grouper(level=-1, freq="D")], sort=False)
    .count()[["currentValue"]]
    .rename(columns={"currentValue": "Changes"})
    .sort_index()
)
df2 = (
    df2.groupby(level=[0, 1], sort=False)
    .cumsum()
    .rename(columns={"Changes": "Cumulative Changes"})
)
df2 = df2.reset_index(level=-1).join(
    df2.reset_index(level=-1)
    .groupby(level=[0, 1], sort=False)[["validFrom"]]
    .first()
    .rename(columns={"validFrom": "First Change"})
)

df2["Days"] = (
    (df2["validFrom"] - df2["First Change"]) / np.timedelta64(1, "D")
).astype(int)
df2.drop(columns=["First Change"], inplace=True)
df2["E"] = df2["Days"] / df2["Cumulative Changes"]
df2.drop(columns=["Days", "Cumulative Changes"], inplace=True)

df2["Next Change"] = df2["validFrom"]
df2.set_index("validFrom", append=True, inplace=True)
df2["Next Change"] = df2["Next Change"].groupby(level=[0, 1], sort=False).shift(-1)
df2["y"] = (
    df2["Next Change"].values - df2.reset_index(level=-1)["validFrom"].values
) / np.timedelta64(1, "D")
df2.drop(columns=["Next Change"], inplace=True)
df2["res"] = df2["y"] - df2["E"]
print(f'Absolute Error: {df2["res"].abs().mean():.1f} days')
print(f'RMSE: {(df2["res"] ** 2).mean() ** 0.5:.1f} days')

df2.head()

In [ ]:
@interact()
def plot_hist2() -> None:
    data = df2.copy()
    data["res %"] = 100 * (data["res"] / data["y"])
    bins = [
        float("-inf"),
        -100,
        -50,
        50,
        100,
        float("inf"),
    ]
    palette = sns.color_palette("icefire", n_colors=len(bins) - 1)
    data["bin"] = pd.cut(data["res %"], bins=bins, right=True, precision=0)

    fig = plt.figure(figsize=(a4[0] * cm, (a4[0] / 2) * cm), dpi=150)

    ax1 = plt.subplot(211)
    sns.scatterplot(
        x="y", y="res", hue="bin", palette=palette, data=data, legend=False, ax=ax1
    )
    sns.despine(ax=ax1, bottom=True)

    ax2 = plt.subplot(212)
    sns.scatterplot(
        x="y", y="E", hue="bin", palette=palette, data=data, legend=False, ax=ax2
    )
    sns.despine(ax=ax2)

    ax1.get_shared_x_axes().join(ax1, ax2)
    ax1.set(xlabel=None, ylabel="Residual")
    ax1.tick_params(axis="x", which="both", bottom=False, top=False, labelbottom=False)
    ax2.set(xlabel="Actual Days to Next Change", ylabel="Predicted Days to Next Change")

    plt.xlim(left=0)

    plt.tight_layout()
    plt.show()